# Fine Tuning Transformers for identification of Human Values behind Arguments

### Introduction

In this notebook we will be fine tuning different transformer models
for the **detection of human values** given a specifc argument. It consists of a **Multilabel text classification** problem where a given piece of text needs to be classified into one or more categories out of the given list. For example, as in this case, a given argument can be categorized into 1 or more human values.

#### Flow of the notebook

The notebook will be divided into seperate sections to provide a organized walk through for the process used. The sections are:

1. [Importing Python Libraries and preparing the environment](#section01)
2. [Importing and Pre-Processing the domain data](#section02)
3. [Preparing the Dataset and Dataloader](#section03)
4. [Creating the Neural Network for Fine Tuning](#section04)
5. [Fine Tuning the Model](#section05)
6. [Validating the Model Performance](#section06)
7. [Evaluating the best model](#section07)
8. [Choosing the best threshold](#section08)
9. [Results](#section09)

#### Technical Details

 - Data: 
	 - We are using the data available on [Zenodo](https://zenodo.org/record/7550385#.Y8wMquzMK3I)
     - [Human Value Detection 2023](https://touche.webis.de/semeval23/touche23-web/index.html) is the competion which provide the souce dataset
	 - We are referring only to the following data: `arguments-training.tsv`, `arguments-validation`, `labels-training.tsv`, `labels-validation.tsv`
	 
     - `arguments-<split>.tsv`: Each row corresponds to one argument
        - `Argument ID`: The unique identifier for the argument
        - `Conclusion`: Conclusion text of the argument
        - `Stance`: Stance of the Premise towards the Conclusion; one of "in favor of", "against"
        - `Premise`: Premise text of the argument

     - `labels-<split>.tsv`: Each row corresponds to one argument
        - `Argument ID`: The unique identifier for the argument
        - `Other`: Each other column corresponds to one value category, with a 1 meaning that the argument resorts to the value category and a 0 that not
---
***NOTE***
- *Since test data are provided without labels, we did not consider them for our analysis. In this regards **the performances of our models have been tested only on validation data**.*
---
  
 - Language Model Used:
	 - BERT BASE
     - BERT LARGE
     - ROBERTA BASE
     - ROBERTA LARGE
     - DISTILBERT
---
***NOTE***

- Auto Models
    - As explained by [Hugging Face doc](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html), in many cases, the architecture we want to use can be guessed from the name or the path of the pretrained model we are supplying to the `from_pretrained` method.
    AutoClasses are here to do this job for us so that we automatically retrieve the relevant model given the name/path to the pretrained weights/config/vocabulary:
        - Instantiating one of `AutoModel` and `AutoTokenizer` will directly create a class of the relevant architecture 
(ex: model = AutoModel.from_pretrained('bert-base-cased') will create a instance of BertModel).
---

 - Hardware Requirements:
	 - Python 3.6 and above
	 - Pytorch, Transformers and All the stock Python ML Libraries
	 - GPU enabled setup 


 - Script Objective:
	 - Fine tuning of different transformer models such that: given a textual argument, classify whether or not the argument draws on one of the following categories:
        * Self-direction: thought      
        * Self-direction: action       
        * Stimulation     
        * Hedonism       
        * Achievement      
        * Power: dominance       
        * Power: resources       
        * Face       
        * Security: personal      
        * Security: societal      
        * Tradition       
        * Conformity: rules       
        * Conformity: interpersonal       
        * Humility       
        * Benevolence: caring       
        * Benevolence: dependability      
        * Universalism: concern       
        * Universalism: nature       
        * Universalism: tolerance      
        * Universalism: objectivity       

---
***NOTE***
- *It is to be noted that the overall mechanisms for a multiclass and multilabel problems are similar, except for few differences namely:*
	- *Loss function is designed to evaluate all the probability of categories individually rather than as compared to other categories. Hence the use of `BCE` rather than `Cross Entropy` when defining loss.*
	- *Sigmoid of the outputs calcuated to rather than Softmax. Again for the reasons defined in the previous point*
	- *The [accuracy metrics](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) and [F1 scores](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score) used from sklearn package as compared to direct comparison of expected vs predicted*
---

<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* Numpy
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* AutoModel and AutoTokenizer

Followed by that we will preapre the device for GPU execeution. This configuration is needed if you want to leverage on onboard GPU.

RANDOM SEED has been settled to 42 to ensure experiments reproducibility.

In [1]:
# Installing the transformers library and additional libraries if looking process 
!pip install -q transformers

In [2]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
best_weights_bert_base = 'drive/MyDrive/Pesi_NLP/bert_base.pth'
best_weights_bert_large = 'drive/MyDrive/Pesi_NLP/bert_large.pth'
best_weights_roberta_base = 'drive/MyDrive/Pesi_NLP/roberta-base.pth'
best_weights_roberta_large = 'drive/MyDrive/Pesi_NLP/roberta_large.pth'
best_weights_distilbert = 'drive/MyDrive/weights_distilbert-base-uncased.pth'

In [5]:
# Random seed to repeat experiments.
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
transformers.set_seed(RANDOM_SEED)

In [6]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Using {device}.')

Using cuda.


<a id='section02'></a>
### Importing and Pre-Processing the domain data

We will be working with the data and preparing for fine tuning purposes, *assuming that the `arguments-train.tsv`, `labels-train.tsv`, `arguments-validation.tsv`, `labels-validation.tsv` are already downloaded and saved in our `data` folder*

* Import files in different dataframes
* Merge arguments and labels dataframe of each corresponding split 
* Taking the values of all the categories and coverting it into a list.
* The list is appened as a new column and other columns are removed

In [7]:
args_train = pd.read_csv("arguments-training.tsv", delimiter='\t')
labels_train = pd.read_csv("labels-training.tsv", delimiter='\t')

In [8]:
df = args_train.merge(labels_train)
df['list'] = df[df.columns[4:]].values.tolist()
df['text'] = df.Conclusion+" "+df.Stance+" "+df.Premise
df_train = df[['text', 'list']].copy()
df_train.head()

,text,list
0,We should ban human cloning in favor of we sho...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,We should ban fast food in favor of fast food ...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,We should end the use of economic sanctions ag...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
3,We should abolish capital punishment against c...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."
4,We should ban factory farming against factory ...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ..."


In [9]:
args_test = pd.read_csv("arguments-validation.tsv", delimiter='\t')
labels_test = pd.read_csv("labels-validation.tsv", delimiter='\t')

In [10]:
df = args_test.merge(labels_test)
df['list'] = df[df.columns[4:]].values.tolist()
df['text'] = df.Conclusion+" "+df.Stance+" "+df.Premise
df_test = df[['text', 'list']].copy()
df_test.head()

,text,list
0,Entrapment should be legalized in favor of if ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,The use of public defenders should be mandator...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Payday loans should be banned in favor of payd...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,Surrogacy should be banned against Surrogacy s...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Entrapment should be legalized against entrapm...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


<a id='section03'></a>
### Preparing the Dataset and Dataloader
We will go through the following workflow:
1. Definition of global variables that will be used later 
during the training/fine tuning stage.
2. Creation of CustomDataset class - This defines how the text is pre-processed before sending it to the neural network. 
3. Definiition of the Dataloader that will feed  the data in batches to the neural network for suitable training and processing. 
---
***NOTE***

Dataset and Dataloader are constructs of the [PyTorch library](https://pytorch.org/docs/stable/data.html) for defining and controlling the data pre-processing and its passage to neural network.


#### Definition of global variables

In [11]:
# Sections of config

# Defining some key variables that will be used later on in the training
#MODEL_NAME = 'bert-base-uncased'
#MODEL_NAME = 'bert-large-uncased'
MODEL_NAME = 'roberta-base'
#MODEL_NAME = 'roberta-large'
#MODEL_NAME = 'distilbert-base-uncased'

MAX_LEN = 75
BATCH_SIZE = 8
EPOCHS = 8
LEARNING_RATE = 2e-05
OUT_CHANNELS = 768 if "base" in  MODEL_NAME else 1024

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


#### *CustomDataset* Dataset Class
- This class is defined to accept the `tokenizer`, `dataframe` and `max_length` as input and generate tokenized output and tags that is used by the BERT/ROBERTA/DISTILBERT model for training. 
- We are using the BERT/ROBERTA/DISTILBERT tokenizer to tokenize the data in the `comment_text` column of the dataframe.
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`, `token_type_ids` (*the latter is not generated by DISTILBERT tokenizer*)

- `targest` is the list of categories labled as `0` or `1` in the dataframe. 
- The *CustomDataset* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model.
- *Validation Dataset* is used to evaluate the performance of the model. 

In [12]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.list #labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [13]:
# Creating the dataset and dataloader for the neural network

print("TRAIN Dataset: {}".format(df_train.shape))
print("TEST Dataset: {}".format(df_test.shape))

training_set = CustomDataset(df_train, tokenizer, MAX_LEN)
testing_set = CustomDataset(df_test, tokenizer, MAX_LEN)

TRAIN Dataset: (5393, 2)
TEST Dataset: (1896, 2)


#### Dataloader
- Dataloader is used to create training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of data loaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [14]:
training_loader = DataLoader(training_set, batch_size = BATCH_SIZE)
testing_loader = DataLoader(testing_set, batch_size = BATCH_SIZE)

<a id='section04'></a>

#### Neural network
We will be creating a neural network with either the `BERTClass` or `DistilbertClass` depending on the established `MODEL_NAME`. 

##### BERTClass

- This network will have the `Bert` / `Roberta` model follwed by a `Droput` and `Linear Layer`. They are added for the purpose of **Regulariaztion** and **Classification** respectively. 
- In the forward loop, there are 2 output from the `BertModel` / `RobertaModel` layer.
- The second output `output_1` or called the `pooled output` is passed to the `Drop Out layer` and the subsequent output is given to the `Linear layer`. 
- Keep note the number of dimensions for `Linear Layer` is **20** because that is the total number of categories in which we are looking to classify our model.
- The data will be fed to the `BertClass` as defined in the dataset. 
- Final layer outputs is what will be used to calcuate the loss and to determine the accuracy of models prediction. 
- We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference. 

#####DistilbertClass
- We will be creating a neural network with the `DistillbertClass`. 
- This network will have the DistilBERT Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs. 
- The data will be fed to the DistilBERT Language model as defined in the dataset. 
- Final layer outputs is what will be compared to the `encoded category` to determine the accuracy of models prediction. 
- We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference. 

---
***NOTE***
- *It is to be noted that the outputs to the BERT and ROBERTA models are different from DISTILBERT Model implemented by the Hugging Face team. There are no `token_type_ids` generated from the tokenizer in case of DISTILBERT and also the final outputs from the network differ.*

---

In [15]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self,model):
        super(BERTClass, self).__init__()
        self.l1 = transformers.AutoModel.from_pretrained(model, return_dict=False)
        self.l2 = torch.nn.Dropout(p=0.3)
        self.l3 = torch.nn.Linear(OUT_CHANNELS, 20)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output


In [16]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilbertClass(torch.nn.Module):
    def __init__(self,model):
        super(DistilbertClass, self).__init__()
        self.l1 = transformers.AutoModel.from_pretrained(model, return_dict=False)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(OUT_CHANNELS, 20)

    def forward(self, ids, mask, token_type_ids):
        output_1 = self.l1(input_ids=ids, attention_mask=mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        
        return output


In [17]:
#torch.cuda.empty_cache()

In [18]:
model = BERTClass(MODEL_NAME) if "distilbert" not in MODEL_NAME else DistilbertClass(MODEL_NAME)
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

#### Loss Function and Optimizer
 - The Loss is defined in the next cell as `loss_fn`.
 - As defined above, the loss function used will be a combination of Binary Cross Entropy which is implemented as [BCELogits Loss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss) in PyTorch
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [19]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [20]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=LEARNING_RATE, 
                  )
                  

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(training_loader)*EPOCHS)


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
#optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

<a id='section05'></a>
### Fine Tuning the Model

Here we define a training function that trains the model on the training dataset created above, specified the number of epochs.

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size. 
- Subsequent output from the model and the actual category are compared to calculate the loss. 
- Loss value is used to optimize the weights of the neurons in the network.
- After every 100 steps the loss value is printed in the console.

In [22]:
def train():
    size = len(training_loader.dataset)
    model.train()
    for batch,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_fn(outputs, targets)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        if batch%100==0:
            current =  batch * len(data['ids'])
            print(f"Train loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")
        
        

<a id='section06'></a>
### Validating the Model

During the validation stage we check the model’s performance against the testing dataset to ensure it is learning. This testing data is the one of `args-validation.tsv`. During the validation stage at each epoch we keep track of the average loss on the validation set. This check is then used to save the best model, such as the model which registered the lowest average loss on validation set during a specific epoch. 

In [23]:
def validation(epoch, val_loss_min_input):
    size = len(testing_loader.dataset)
    num_batches = len(testing_loader)
    model.eval()
    val_loss = 0
    #fin_targets=[]
    #fin_outputs=[]
    with torch.no_grad():
        for batch_idx, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            val_loss += loss_fn(outputs, targets).item()
        
        val_loss /= num_batches
        #outputs, targets = fin_outputs, fin_targets
        print(f"\nValidation loss: {val_loss:>8f}.")
        ## TODO: save the model if validation loss has decreased
        if val_loss <= val_loss_min_input:
            #create checkpoint variable and add important data
            if epoch > 0: 
                print('Validation loss decreased ({:.8f} --> {:.8f}).  Saving model ...'.format(val_loss_min_input, val_loss))
            else: print('Saving model ...')   
            # save best moel
            torch.save(model.state_dict(), f"weights_{MODEL_NAME}.pth")
            print(f"Saved PyTorch Model State to weights_{MODEL_NAME}.pth\n")
            val_loss_min_input = val_loss
    
    return val_loss_min_input
       

In [24]:
val_loss_min = np.inf
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train()
    val_loss_min = validation(epoch, val_loss_min)
   

Epoch 1
-------------------------------
Train loss: 0.686985  [    0/ 5393]
Train loss: 0.449704  [  800/ 5393]
Train loss: 0.425893  [ 1600/ 5393]
Train loss: 0.353953  [ 2400/ 5393]
Train loss: 0.290618  [ 3200/ 5393]
Train loss: 0.351458  [ 4000/ 5393]
Train loss: 0.356625  [ 4800/ 5393]

Validation loss: 0.352989.
Saving model ...
Saved PyTorch Model State to weights_roberta-base.pth

Epoch 2
-------------------------------
Train loss: 0.268978  [    0/ 5393]
Train loss: 0.329695  [  800/ 5393]
Train loss: 0.326498  [ 1600/ 5393]
Train loss: 0.273389  [ 2400/ 5393]
Train loss: 0.244819  [ 3200/ 5393]
Train loss: 0.308570  [ 4000/ 5393]
Train loss: 0.328398  [ 4800/ 5393]

Validation loss: 0.336518.
Validation loss decreased (0.35298883 --> 0.33651756).  Saving model ...
Saved PyTorch Model State to weights_roberta-base.pth

Epoch 3
-------------------------------
Train loss: 0.206115  [    0/ 5393]
Train loss: 0.285926  [  800/ 5393]
Train loss: 0.289492  [ 1600/ 5393]
Train loss: 

# MODELS EVALUATION

In [25]:
def test(loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

## BERT BASE

### Restoring the best model
According to the average validation loss calculated at epoch, we restored the weights of the best model saved into `model.pth`. 

In [26]:
# restore best model 
if MODEL_NAME == 'bert-base-uncased':
    model = BERTClass(MODEL_NAME) if "distilbert" not in MODEL_NAME else DistilbertClass(MODEL_NAME)
    model.load_state_dict(torch.load(f'weights_{MODEL_NAME}.pth'))
    model = model.to(device)

<a id='section07'></a>
### Evaluating the best model
To get a measure of our model performance we are mainly taking into account
the F1 Macro score since is the best indicator in case of unbalanced classes as the current case.

In [27]:
if MODEL_NAME == 'bert-base-uncased':
    outputs, targets = test(testing_loader)

<a id='section08'></a>
### Choosing the best threshold
Since the outoput scores are probabilities values in the range [0,1] we need to define the best threshold to approximate each score either to 0 or to 1. 
To do that, we considered all possible threshold values between 0.1 and 0.85 with steps of 0.05 such that: all the values higher than or equal to the threshold are approximated to 1, otherwise to 0.
Finally, we take the threshold value which gave us the highest F1 score.

In [28]:
if MODEL_NAME == 'bert-base-uncased':
    results = {}
    for tr in np.arange(0.1, 0.9, 0.05):
        tr = round(tr,2)
        predictions = np.array(outputs) >= tr
        f1 = f1_score(targets, predictions, average = "macro", zero_division = 1)
        results[tr] = f1

    for k,v in results.items():
        print("THRESHOLD: {:.2f} ".format(k), "F1 score: {:.3f}".format(v))

In [29]:
if MODEL_NAME == 'bert-base-uncased':
    THRESHOLD = max(results, key = results.get)
    print("BEST THRESHOLD:", THRESHOLD , "with F1 score: {:.2f}".format(max(results.values())))

<a id='section09'></a>
### Classification Report

In [30]:
if MODEL_NAME == 'bert-base-uncased':
    predictions = np.array(outputs) >= THRESHOLD
    labels = labels_test.columns[1:]
    print(classification_report(predictions, targets, target_names = labels, zero_division=1))

## BERT LARGE

In [31]:
# restore best model 
if MODEL_NAME == 'bert-large-uncased':
    model = BERTClass(MODEL_NAME) if "distilbert" not in MODEL_NAME else DistilbertClass(MODEL_NAME)
    model.load_state_dict(torch.load('model.pth'))
    model = model.to(device)

In [32]:
if MODEL_NAME == 'bert-large-uncased':
    outputs, targets = test(testing_loader)

In [33]:
if MODEL_NAME == 'bert-large-uncased':
    results = {}
    for tr in np.arange(0.1, 0.9, 0.05):
        tr = round(tr,2)
        predictions = np.array(outputs) >= tr
        f1 = f1_score(targets, predictions, average = "macro", zero_division = 1)
        results[tr] = f1

    for k,v in results.items():
        print("THRESHOLD: {:.2f} ".format(k), "F1 score: {:.3f}".format(v))

In [34]:
if MODEL_NAME == 'bert-large-uncased':
    THRESHOLD = max(results, key = results.get)
    print("BEST THRESHOLD:", THRESHOLD , "with F1 score: {:.2f}".format(max(results.values())))

In [35]:
if MODEL_NAME == 'bert-large-uncased':
    predictions = np.array(outputs) >= THRESHOLD
    labels = labels_test.columns[1:]
    print(classification_report(predictions, targets, target_names = labels, zero_division=1))

## ROBERTA BASE 

In [36]:
# restore best model 
if MODEL_NAME == 'roberta-base':
    model = BERTClass(MODEL_NAME) if "distilbert" not in MODEL_NAME else DistilbertClass(MODEL_NAME)
    model.load_state_dict(torch.load(f'weights_{MODEL_NAME}.pth'))
    model = model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
if MODEL_NAME == 'roberta-base':
    outputs, targets = test(testing_loader)

In [38]:
if MODEL_NAME == 'roberta-base':
    results = {}
    for tr in np.arange(0.1, 0.9, 0.05):
        tr = round(tr,2)
        predictions = np.array(outputs) >= tr
        f1 = f1_score(targets, predictions, average = "macro", zero_division = 1)
        results[tr] = f1

    for k,v in results.items():
        print("THRESHOLD: {:.2f} ".format(k), "F1 score: {:.3f}".format(v))

THRESHOLD: 0.10  F1 score: 0.450
THRESHOLD: 0.15  F1 score: 0.474
THRESHOLD: 0.20  F1 score: 0.472
THRESHOLD: 0.25  F1 score: 0.460
THRESHOLD: 0.30  F1 score: 0.442
THRESHOLD: 0.35  F1 score: 0.428
THRESHOLD: 0.40  F1 score: 0.413
THRESHOLD: 0.45  F1 score: 0.399
THRESHOLD: 0.50  F1 score: 0.383
THRESHOLD: 0.55  F1 score: 0.370
THRESHOLD: 0.60  F1 score: 0.359
THRESHOLD: 0.65  F1 score: 0.343
THRESHOLD: 0.70  F1 score: 0.323
THRESHOLD: 0.75  F1 score: 0.306
THRESHOLD: 0.80  F1 score: 0.283
THRESHOLD: 0.85  F1 score: 0.253


In [39]:
if MODEL_NAME == 'roberta-base':
    THRESHOLD = max(results, key = results.get)
    print("BEST THRESHOLD:", THRESHOLD , "with F1 score: {:.2f}".format(max(results.values())))

BEST THRESHOLD: 0.15 with F1 score: 0.47


In [40]:
if MODEL_NAME == 'roberta-base':
    predictions = np.array(outputs) >= THRESHOLD
    labels = labels_test.columns[1:]
    print(classification_report(predictions, targets, target_names = labels, zero_division=1))

                            precision    recall  f1-score   support

   Self-direction: thought       0.75      0.36      0.49       520
    Self-direction: action       0.73      0.46      0.56       790
               Stimulation       0.26      0.38      0.31        96
                  Hedonism       0.35      0.46      0.40        78
               Achievement       0.81      0.52      0.63       897
          Power: dominance       0.39      0.27      0.32       234
          Power: resources       0.68      0.35      0.46       258
                      Face       0.32      0.30      0.31       135
        Security: personal       0.93      0.58      0.71      1228
        Security: societal       0.80      0.50      0.61       778
                 Tradition       0.47      0.42      0.44       194
         Conformity: rules       0.72      0.48      0.58       673
 Conformity: interpersonal       0.27      0.40      0.32        40
                  Humility       0.21      0.15

## ROBERTA LARGE

In [41]:
# restore best model 
if MODEL_NAME == 'roberta-large':
    model = BERTClass(MODEL_NAME) if "distilbert" not in MODEL_NAME else DistilbertClass(MODEL_NAME)
    model.load_state_dict(torch.load(f'weights_{MODEL_NAME}.pth'))
    model = model.to(device)

In [42]:
if MODEL_NAME == 'roberta-large':
    outputs, targets = test(training_loader)

In [43]:
if MODEL_NAME == 'roberta-large':
    results = {}
    for tr in np.arange(0.1, 0.9, 0.05):
        tr = round(tr,2)
        predictions = np.array(outputs) >= tr
        f1 = f1_score(targets, predictions, average = "macro", zero_division = 1)
        results[tr] = f1

    for k,v in results.items():
        print("THRESHOLD: {:.2f} ".format(k), "F1 score: {:.3f}".format(v))

In [44]:
if MODEL_NAME == 'roberta-large':
    THRESHOLD = max(results, key = results.get)
    print("BEST THRESHOLD:", THRESHOLD , "with F1 score: {:.2f}".format(max(results.values())))

In [45]:
if MODEL_NAME == 'roberta-large':
    predictions = np.array(outputs) >= THRESHOLD
    labels = labels_test.columns[1:]
    print(classification_report(targets, predictions, target_names = labels, zero_division=1))

In [46]:
if MODEL_NAME == 'roberta-large':
    outputs, targets = test(testing_loader)

In [47]:
if MODEL_NAME == 'roberta-large':
    results = {}
    for tr in np.arange(0.1, 0.9, 0.05):
        tr = round(tr,2)
        predictions = np.array(outputs) >= tr
        f1 = f1_score(targets, predictions, average = "macro", zero_division = 1)
        results[tr] = f1

    for k,v in results.items():
        print("THRESHOLD: {:.2f} ".format(k), "F1 score: {:.3f}".format(v))

In [48]:
if MODEL_NAME == 'roberta-large':
    THRESHOLD = max(results, key = results.get)
    print("BEST THRESHOLD:", THRESHOLD , "with F1 score: {:.2f}".format(max(results.values())))

In [49]:
if MODEL_NAME == 'roberta-large':
    predictions = np.array(outputs) >= THRESHOLD
    labels = labels_test.columns[1:]
    print(classification_report(targets, predictions, target_names = labels, zero_division=1))

## DISTILBERT

In [50]:
# restore best model 
if MODEL_NAME == 'distilbert-base-uncased':
    model = BERTClass(MODEL_NAME) if "distilbert" not in MODEL_NAME else DistilbertClass(MODEL_NAME)
    model.load_state_dict(torch.load(f'weights_{MODEL_NAME}.pth'))
    model = model.to(device)

In [51]:
if MODEL_NAME == 'distilbert-base-uncased':
    outputs, targets = test(testing_loader)

In [52]:
if MODEL_NAME == 'distilbert-base-uncased':
    results = {}
    for tr in np.arange(0.1, 0.9, 0.05):
        tr = round(tr,2)
        predictions = np.array(outputs) >= tr
        f1 = f1_score(targets, predictions, average = "macro", zero_division = 1)
        results[tr] = f1

    for k,v in results.items():
        print("THRESHOLD: {:.2f} ".format(k), "F1 score: {:.3f}".format(v))

In [53]:
if MODEL_NAME == 'distilbert-base-uncased':
    THRESHOLD = max(results, key = results.get)
    print("BEST THRESHOLD:", THRESHOLD , "with F1 score: {:.2f}".format(max(results.values())))

In [54]:
if MODEL_NAME == 'distilbert-base-uncased':
    predictions = np.array(outputs) >= THRESHOLD
    labels = labels_test.columns[1:]
    print(classification_report(predictions, targets, target_names = labels, zero_division=1))